# Création fichier csv clean

Ce notebook est consacré à la création du fichier qui sera utilisé comme base de données. Pour cela, nou savons pris le format CSV, avec comme séparateur la barre verticale "|" 
Il est séparé en plusieurs étapes : 

    1) Installations et importations
    2) Fonctions utilitaires
    3) Script permettant l'écriture du fichier
    

### 1) Installations et importations {#petit1}

Installations pour la récupération des données


In [ ]:
#Installations pour la récupération des données

%pip install webdriver-manager
%pip install -q lxml
%pip install bs4

Installations de spacy et du package des mots français inutiles pour le cleaning des questions

In [ ]:
#Installation pour le cleaning

%pip install spacy 
%python -m spacy download fr_core_news_sm

In [2]:
#imports

import pandas as pd
import bs4
import lxml
import pandas
import urllib
import re

from urllib import request

import spacy

### 2) Fonctions utilitaires

In [3]:
#Fonctions utilitaires

def pageurl(x) : 
    return "https://questions.assemblee-nationale.fr/q16/16-"+str(x)+"QE.htm"


nlp = spacy.load('fr_core_news_sm')

def clean_docs(texts, ministere, remove_stopwords=True, n_process = 4):
    
    docs = nlp.pipe(texts, 
                    n_process=n_process,
                    disable=['parser', 'ner', 'textcat'])
    stopwords = nlp.Defaults.stop_words.union(set(['m.','mme','ministre', ministere.lower()]))

    docs_cleaned = []
    for doc in docs:
        tokens = [tok.text.lower().strip() for tok in doc if not tok.is_punct]
        if remove_stopwords:
            tokens = [tok for tok in tokens if tok not in stopwords]
        doc_clean = ' '.join(tokens)
        docs_cleaned.append(doc_clean)
        
    return docs_cleaned

# pas sur de l'utilité : mots qui peuvent etre quand meme utile (souveraineté ?), appelle parfois le délégué des finances alors qu'interroge le ministere des comptes publics
"""def clean_question2(text, ministere, nom, remove_stopwords=True, n_process=4):
    
    text = nlp(text)

    inclus = ministere.lower().split() + nom.lower().split()
    print(inclus)
    stopwords = nlp.Defaults.stop_words 
    stopwords = stopwords.union(set(['m.','mme','ministre']+inclus))
    
    tokens = [tok.text.lower().strip() for tok in text if not tok.is_punct]
    if remove_stopwords:
        tokens = [tok for tok in tokens if tok not in stopwords]
    question_clean = ' '.join(tokens)
    return question_clean"""

def clean_question(text, nom, remove_stopwords=True, n_process=4):
    
    text = nlp(text)

    stopwords = nlp.Defaults.stop_words 
    stopwords = stopwords.union(set(['m.','mme','ministre','été', 'cas', 'ans',"jour","appeller", "attention", "attirer", "non", "pourtant"]+nom.lower().split()))
    
    tokens = [tok.lemma_ for tok in text if not tok.is_punct]
    if remove_stopwords:
        tokens = [tok for tok in tokens if tok not in stopwords]
    question_clean = ' '.join(tokens)
    return question_clean



In [76]:
import spacy

# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")

# Texte à analyser
texte = "Votre texte ici. Il peut contenir plusieurs phrases. j'ai mangé beaucoup de gateaux. j'ai été sage cet été"

texte = "M. Christophe Naegelen appelle l'attention de M. le ministre de l'agriculture et de la souveraineté alimentaire sur le statut particulier des « patous » communément appelés chiens de berger ainsi que sur les dispositifs mis en place pour permettre la cohabitation entre les randonneurs et les patous. Les chiens de berger protègent les troupeaux et ne sont pas considérés comme divaguant lorsqu'ils se trouvent sans laisse et à distance de leur maître depuis la loi n° 2005-157 du 23 février 2005 qui a modifié l'article L. 211-23 du code rural. Désormais, la loi libère « de toute poursuite pénale pour divagation, l'éleveur ou le propriétaire d'un chien affecté à la protection du troupeau (« patous ») ou à la garde de celui-ci (chien de conduite) ». Des études ont montré que les patous ne sont pas dangereux pour les hommes, ces chiens sont dressés pour dissuader et non pour blesser. Néanmoins, des cas d'agressions sont recensés et une forte part des randonneurs sont inquiets de croiser ces chiens. Des panneaux d'informations sont présents en montagne pour prévenir les promeneurs et leur donner les consignes à suivre en cas de présence des « patous ». Cependant, la période estivale qui arrive laisse présager une forte affluence de randonneurs dans les Hautes-Vosges ainsi que dans tous les massifs français. C'est pourquoi intensifier la prévention et l'installation de panneaux d'information dès lors que les randonneurs entrent dans une zone dans laquelle les « patous » sont présents semble être opportun. Il l'interpelle sur le développement de dispositifs permettant une meilleure cohabitation des randonneurs et des « patous », afin d'articuler les problématiques touchant à la protection des agriculteurs et celles relatives la protection de l'environnement."



# Analyser le texte avec spaCy
doc = nlp(texte)



"""def clean_question(text, nom, remove_stopwords=True, n_process=4):
    text = nlp(text)

    stopwords = nlp.Defaults.stop_words 
    stopwords = stopwords.union(set(['m.','mme','ministre', 'cas', 'ans', "interroger", "attire attention", "non", "pourtant"]+nom.lower().split()))
    
    # Utiliser lemma_ pour obtenir le lemme du mot
    tokens = [tok.lemma_ for tok in text if not tok.is_punct]
    if remove_stopwords:
        tokens = [tok for tok in tokens if tok not in stopwords]
    question_clean = ' '.join(tokens)
    return question_clean
"""

def clean_question(text, nom, remove_stopwords=True, n_process=4):
    text = nlp(text)

    stopwords = nlp.Defaults.stop_words 
    stopwords = stopwords.union(set(['m.','mme','ministre','été', 'cas', 'ans', "non", "pourtant"]+nom.lower().split()))

    # Liste des expressions à exclure
    expressions_a_exclure = ["appelle l'attention de", "attire l'attention de"]

    # Utiliser lemma_ pour obtenir le lemme du mot
    tokens = [tok.lemma_ if tok.pos_ != "VERB" and tok.text.lower() not in expressions_a_exclure else tok.lemma_ for tok in text if not tok.is_punct]
    if remove_stopwords:
        tokens = [tok for tok in tokens if tok not in stopwords]
    question_clean = ' '.join(tokens)
    return question_clean




# Extraire et afficher tous les verbes à l'infinitif
verbes_infinitif = [token.lemma_ for token in doc if token.pos_ == "VERB"]
print(verbes_infinitif)
print(clean_question(texte,"fabrice"))

['appeler', 'appeler', 'berger', 'mettre', 'permettre', 'berger', 'protéger', 'considérer', 'divaguer', 'trouver', 'laisse', 'modifier', 'libérer', 'affecter', '«', 'montrer', 'dresser', 'dissuader', 'blesser', 'recenser', 'croiser', 'prévenir', 'donner', 'suivre', 'arriver', 'laisse', 'présager', 'être', 'intensifier', 'sembler', 'opportun', 'permettre', 'articuler', 'toucher']
Christophe Naegelen appeler attention agriculture souveraineté alimentaire statut particulier patous communément appeler chien berger dispositif mettre place permettre cohabitation entrer randonneur patou chien berger protéger troupeau considérer divaguer trouver laisse distance maître loi numéro 2005 157 23 février 2005 modifier article L. 211 23 code rural loi libérer poursuite pénal divagation éleveur propriétaire chien affecter protection troupeau patous garde chien conduite étude montrer patou dangereux homme chien dresser dissuader blesser agression recenser fort part randonneur inquiet croiser chien pann

In [79]:
#création fichier

données = open("data_cleaned.csv", "w", encoding="utf-8")

données.write("id|groupe|nom|ministère_interrogé|rubrique|titre|date|question|question_clean")

données.close()

In [4]:
données = open("data_cleaned.csv", "a", encoding="utf-8")

for k in range(169, 1500) : 
    print(k)
    source_code = request.urlopen(pageurl(k)).read()
    page = bs4.BeautifulSoup(source_code)
    question0 = page.find("div", {"class" : "question"})
    question = question0.find("p")
    question = question.text.strip()
    
    rubrique0 = page.findAll("div", {"class" : "question_col33"})
    rubrique,titre = rubrique0[0].find("p"), rubrique0[1].find("p")
    rubrique, titre = rubrique.text.strip()[11:], titre.text.strip()[8:]
    
    ministere0 = page.find("div", {"class" : "question_col50"})
    ministere = ministere0.text.strip()[23:]

    lien0 = page.find("span", {"class" : "question_big_content"})
    lien = lien0.find("a").get("href")

    date0 = page.find("div", {"class" : "question_publish_date"})
    date = date0.find("span")
    date = date.text.strip()
    
    source_code_parlementaire = request.urlopen(lien).read()
    page_parlementaire = bs4.BeautifulSoup(source_code_parlementaire)

    groupe0 = page_parlementaire.find("a", {"class" : "h4 _colored link"})
    groupe = groupe0.text.strip()

    nom0 = page_parlementaire.find("h1", {"class" : "h1 _mt-small"})
    nom = nom0.text.strip()

    question_clean = clean_question(question, nom)
    print(str(k)+ "oui")

    données.write("\n" + str(k) + "|" + groupe + "|" + nom +  "|" + ministere + "|" + rubrique + "|" + titre + "|" + date + "|" + question + "|" + question_clean)

données.close()

169
169oui
170
170oui
171
171oui
172
172oui
173
173oui
174
174oui
175
175oui
176
176oui
177
177oui
178
178oui
179
179oui
180
180oui
181


KeyboardInterrupt: 